In [1]:
import json
import os
from pathlib import Path
script_dir = os.path.dirname(Path())
data_snippets = json.load(open(os.path.join(script_dir, "data/snippets.txt")), encoding='utf-8')

In [2]:
for idx, arguments in enumerate(data_snippets):
    arguments['index'] = idx
    # print(arguments)

# removing arguments with sentences less then 3
print(len(data_snippets))
count = 0
data_snippets_filtered = []
for argument_x in data_snippets:

    if len(argument_x['sentences']) >= 3:
        data_snippets_filtered.append(argument_x)
        count = count + 1

print(count)

100
77


In [3]:
argument_query_list =[]
for arguments in data_snippets_filtered:
    if arguments['query'] not in argument_query_list:
        argument_query_list.append(arguments['query'])

In [4]:
len(argument_query_list)

10

In [5]:
print(argument_query_list)

['abortion', 'brexit', 'climate change', 'death_penalty', 'donald trump', 'feminism', 'google', 'nuclear_energy', 'trump', 'vegan']


In [6]:
arguments_query_count =  {}
for query in argument_query_list:
    
    arguments_count = 0
    for argument in data_snippets_filtered: 
        if argument['query']== query:
            arguments_count = arguments_count +1
        
        arguments_query_count[query]= arguments_count
    
argument_count_list=arguments_query_count  

In [7]:
print(argument_count_list)

{'abortion': 10, 'brexit': 9, 'climate change': 6, 'death_penalty': 9, 'donald trump': 10, 'feminism': 9, 'google': 8, 'nuclear_energy': 3, 'trump': 5, 'vegan': 8}


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
dev_args_set = ['feminism','death_penalty', 'brexit']

In [10]:
test_args =[]
dev_args  =[]
for args in data_snippets_filtered:
    if args['query'] in dev_args_set:
        dev_args.append(args)
    else:
        test_args.append(args)

In [11]:
len(test_args)

50

In [12]:
len(dev_args)

27

In [18]:
# setup.py

In [17]:
import nltk
import tensorflow_hub as hub
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from fastai.text.learner import load_learner

nltk.download('punkt')
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding", "Agree?"])

print(embeddings)
tokenizer = AutoTokenizer.from_pretrained("chkla/roberta-argument")
arg_model = AutoModelForSequenceClassification.from_pretrained("chkla/roberta-argument")
print(arg_model.config)
model_path = "../pretrained_models2"
claim_classifier = load_learner(model_path)
print(claim_classifier)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:absl:Using C:\Users\harsh\AppData\Local\Temp\tfhub_modules to cache modules.


tf.Tensor(
[[-0.03133  -0.063386 -0.016075 -0.010349 ...  0.064889 -0.032428 -0.045757  0.053705]
 [ 0.050809 -0.016524  0.015738 -0.042864 ... -0.01628   0.009767  0.031701  0.017881]
 [ 0.053329 -0.017461  0.053533  0.057924 ... -0.007853 -0.022668 -0.040348 -0.035826]], shape=(3, 512), dtype=float32)
RobertaConfig {
  "_name_or_path": "chkla/roberta-argument",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NON-ARGUMENT",
    "1": "ARGUMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ARGUMENT": 1,
    "NON-ARGUMENT": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type

RNNLearner(data=TextClasDataBunch;

Train: LabelList (0 items)
x: TextList

y: CategoryList

Path: ..\pretrained_models2;

Valid: LabelList (0 items)
x: TextList

y: CategoryList

Path: ..\pretrained_models2;

Test: None, model=SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60000, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60000, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200

In [15]:
from snippetGenerator import SnippetGenerator

In [16]:
def evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method):
    snippetGenerator= SnippetGenerator(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
                                     argument_context, argumentative_score_method)
    snippets = snippetGenerator.get_snippets(arguments)
    count, accuracy= snippetGenerator.get_accuracy(arguments, snippets)
    return count,accuracy

# Test Snippet Generation using different parameters --

1. Previous Vesion of the snippet Generation

In [70]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0.15
mc_method = 'linear'
aspects_arguments_max = 0
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [0, 1, 0]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [0, 1, 0]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0.15


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
18.5
Accuracy  37.0


(18.5, 37.0)

In [71]:
# setting d= 0 for evaluation of argument context modelling task

In [72]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0
mc_method = 'linear'
aspects_arguments_max = 0
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [0, 1, 0]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [0, 1, 0]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
18.5
Accuracy  37.0


(18.5, 37.0)

In [74]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0
mc_method = 'linear'
aspects_arguments_max = 0
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 1, 0]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 1, 0]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
19.5
Accuracy  39.0


(19.5, 39.0)

In [79]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0
mc_method = 'linear'
aspects_arguments_max = 50
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 0, 1]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 50
snippets generated File created 
21.0
Accuracy  42.0


(21.0, 42.0)

In [80]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0
mc_method = 'linear'
aspects_arguments_max = 50
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 1, 1]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 1, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 50
snippets generated File created 
20.5
Accuracy  41.0


(20.5, 41.0)

In [81]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 0
mc_method = 'linear'
aspects_arguments_max = 100
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 1, 1]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 1, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 100
snippets generated File created 
19.5
Accuracy  39.0


(19.5, 39.0)

## 2.Evaluation on  number of aspect generated arguments

In [82]:
d = 0
# methodSet = ['power','eigen','linear','krylov']
mc_method = 'linear'
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
count_list =[]
accuracy_list =[]

aspects_weights = [0, 0]
arguments = test_args
aspects_arguments_max_list = [0,50,100,150,200,250,300]
argumentative_score_method = argumentative_score_methods[0]
argument_context =[1,0,1]

accuracy_list_aspect = []
count_list_aspect =[]


for aspects_arguments_max in aspects_arguments_max_list:
    count,accuracy =evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)
    accuracy_list_aspect.append(accuracy)
    count_list_aspect.append(count)
    

Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
19.5
Accuracy  39.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 50
snippets generated File created 
21.0
Accuracy  42.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 100
snippets generated File created 
21.0
Accuracy  42.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 150
snippets generated File created 
22.0
Accuracy  44.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 200
snippets generated File created 
22.5
Accuracy  45.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 250
snippets generated File created 
21.5
Accuracy  43.0
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 300
snippets generated File created 
21.0
Accuracy  42.0


In [85]:
import pandas as pd
evaluation_aspect_detection_test = pd.DataFrame(
    {'Number of Arguemts': aspects_arguments_max_list,
     'match count': accuracy_list_aspect,
     'accuracy (%)': count_list_aspect
    })

In [86]:
evaluation_aspect_detection_test

,Number of Arguemts,match count,accuracy (%)
0,0,19.5,39.0
1,50,21.0,42.0
2,100,21.0,42.0
3,150,22.0,44.0
4,200,22.5,45.0
5,250,21.5,43.0
6,300,21.0,42.0


In [88]:
evaluation_aspect_detection_test.to_csv('data/evaluation_aspect_detection_test')

In [61]:
d = 0
# methodSet = ['power','eigen','linear','krylov']
mc_method = 'linear'
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
count_list =[]
accuracy_list =[]

aspects_weights = [0, 0]
arguments = dev_args
# aspects_arguments_max_list = [0,50,100,150,200,250,300]
argumentative_score_method = argumentative_score_methods[0]
argument_context =[1,0,1]

accuracy_list_aspect = []
count_list_aspect =[]


for aspects_arguments_max in aspects_arguments_max_list:
    accuracy, count =evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)
    accuracy_list_aspect.append(accuracy)
    count_list_aspect.append(count)
    

Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 0


snippets generated File created 
8.5
Accuracy  31.48148148148148
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 50
snippets generated File created 
9.5
Accuracy  35.18518518518518
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 100
snippets generated File created 
10.5
Accuracy  38.888888888888886
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 150
snippets generated File created 
12.0
Accuracy  44.44444444444444
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 200
snippets generated File created 
12.5
Accuracy  46.2962962962963
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 250
snippets generated File created 
11.5
Accuracy  42.592592592592595
Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 0


aspects_weights: [0, 0] aspects_arguments_max 300
snippets generated File created 
11.5
Accuracy  42.592592592592595


## Selecting d=1 to test argumentatative score method

In [90]:

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 1
mc_method = 'linear'
aspects_arguments_max = 0
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [0, 0, 0]
argumentative_score_method = argumentative_score_methods[0]

evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [0, 0, 0]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 1


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
17.5
Accuracy  35.0


(17.5, 35.0)

In [20]:
accuracy_list_args_method = []
count_list_args_method = []

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 1
mc_method = 'linear'
aspects_arguments_max = 0
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [0, 0, 0]
#argumentative_score_method = argumentative_score_methods[0]
for argumentative_score_method in argumentative_score_methods:
    count, accuracy =evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)
    accuracy_list_args_method.append(accuracy)
    count_list_args_method.append(count)

Context_array [0, 0, 0]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 1


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 
17.5
Accuracy  35.0
Context_array [0, 0, 0]
Markov Chain Method linear Argumentative Score Method argument_score d: 1


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 


19.0
Accuracy  38.0
Context_array [0, 0, 0]
Markov Chain Method linear Argumentative Score Method claim_score d: 1


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 


14.5
Accuracy  29.0
Context_array [0, 0, 0]
Markov Chain Method linear Argumentative Score Method hybrid_score d: 1


aspects_weights: [0, 0] aspects_arguments_max 0
snippets generated File created 


15.5
Accuracy  31.0


In [22]:
import pandas as pd
evaluation_argumentation_methods_test = pd.DataFrame(
    {'Argumentataion-score-methods': argumentative_score_methods,
     'match count': accuracy_list_args_method,
     'accuracy (%)': count_list_args_method
    })

In [23]:
evaluation_argumentation_methods_test

,Argumentataion-score-methods,match count,accuracy (%)
0,discourse_claim_markers,35.0,17.5
1,argument_score,38.0,19.0
2,claim_score,29.0,14.5
3,hybrid_score,31.0,15.5


In [25]:
accuracy_list_args_method = []
count_list_args_method = []

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 1
mc_method = 'linear'
aspects_arguments_max = 200
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 0, 1]
argumentative_score_method =argumentative_score_methods[0]
evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method discourse_claim_markers d: 1


aspects_weights: [0, 0] aspects_arguments_max 200
snippets generated File created 


16.0
Accuracy  32.0


(16.0, 32.0)

In [26]:
accuracy_list_args_method = []
count_list_args_method = []

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 1
mc_method = 'linear'
aspects_arguments_max = 200
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 0, 1]
argumentative_score_method =argumentative_score_methods[1]
evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)


Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method argument_score d: 1


aspects_weights: [0, 0] aspects_arguments_max 200
snippets generated File created 


19.0
Accuracy  38.0


(19.0, 38.0)

In [ ]:
accuracy_list_args_method = []
count_list_args_method = []

# methodSet = ['power','eigen','linear','krylov']
argumentative_score_methods = ['discourse_claim_markers', 'argument_score', 'claim_score', 'hybrid_score']
d = 1
mc_method = 'linear'
aspects_arguments_max = 200
aspects_weights = [0, 0]
arguments = test_args
# argument_context =[1,1,1]
# argument_context_clusters = ['query',same page','aspect']
argument_context = [1, 0, 1]
argumentative_score_method =argumentative_score_methods[2]
evaluation(arguments, d, mc_method, aspects_arguments_max, aspects_weights,
               argument_context, argumentative_score_method)

Context_array [1, 0, 1]
Markov Chain Method linear Argumentative Score Method claim_score d: 1


aspects_weights: [0, 0] aspects_arguments_max 200
